# 7. Visualization Using TensorBoard
Computation graphs are hard to imagine. The progress of training is also hard to imagine. Let's visualize them. Fortunately, TensorFlow makes this very very easy with TensorBoard. In any TensorFlow sessions, you can save the computation graph and the summary statistics (e.g. loss, accuracy) you would like to track. These will be saved in a log folder. Tensorboard loads the files in the log folder and visualize them nicely. 

In [17]:
import tensorflow as tf

In the section 4, we had the following code:

In [18]:
a = tf.add(2,3)       

with tf.Session() as sess:
    print('a:',sess.run(a))

a: 5


Let's rewrite this code using more TensorFlow:

In [19]:
a = tf.constant(2, name = 'a')
b = tf.constant(3, name = 'b')
sum_ = tf.add(a,b, name = 'sum_')       

with tf.Session() as sess:
    print('sum_:',sess.run(sum_))

sum_: 5


## Basic Visualization

How does the computation graph of this code look like? Let's visualize it using TensorBoard. To do this, you just need to add one line to the code:

In [20]:
tf.reset_default_graph() # Clearing all tensors before this
a = tf.constant(2, name = 'a')
b = tf.constant(3, name = 'b')
sum_ = tf.add(a,b, name = 'sum_')       

with tf.Session() as sess:
    print('sum_:',sess.run(sum_))
    writer = tf.summary.FileWriter('log/simple', sess.graph) # NEW!
    writer.close()

sum_: 5


This should create a new directory called `log` to where this Jupyter Notebook is located. Now, run the following on your local terminal, 

`tensorboard --logdir=log/simple`

Then, head to `http://localhost:6006` and see for yourself that the graph is being visualized!

## Logistic Regression Visualization

### Simple Visualization

In [21]:
tf.reset_default_graph() # Clearing all tensors before this

In [22]:


# Import MINST data
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets('MNIST_data', one_hot=True)

# Variables
M = 784
K = 10
X = tf.placeholder(tf.float32, shape=[None, M])
Y = tf.placeholder(tf.float32, shape=[None, K])
W = tf.Variable(tf.zeros([M, K]))
b = tf.Variable(tf.zeros([K]))

# Model
Yhat = tf.nn.softmax(tf.matmul(X, W) + b)

# Loss
loss = tf.reduce_mean(-tf.reduce_sum(Y*tf.log(Yhat), axis=1))

# Optimizer
learning_rate = 0.01
optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss)

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [23]:
# Train
num_epochs  = 25
batch_size  = 100

with tf.Session() as sess:
    writer = tf.summary.FileWriter('log/logistic1', sess.graph) # NEW!
    writer.close()

### Better: Name the variables

In [24]:
tf.reset_default_graph() # Clearing all tensors before this

In [25]:


# Import MINST data
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets('MNIST_data', one_hot=True)

# Variables
M = 784
K = 10
X = tf.placeholder(tf.float32, shape=[None, M], name='Input-Images')
Y = tf.placeholder(tf.float32, shape=[None, K], name='Output-Labels')
W = tf.get_variable('Weights', initializer=tf.zeros([M, K]))
b = tf.get_variable('Bias', initializer=tf.zeros([K]))

# Model
Yhat = tf.nn.softmax(tf.matmul(X, W) + b, name='Prediction-Labels')

# Loss
loss = tf.reduce_mean(-tf.reduce_sum(Y*tf.log(Yhat), axis=1), name="Cross-Entropy-Loss")

# Optimizer
learning_rate = 0.01
optimizer = tf.train.GradientDescentOptimizer(learning_rate, name="SGD-Optimizer")
update    = optimizer.minimize(loss)

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [26]:
# Train
num_epochs  = 25
batch_size  = 100

with tf.Session() as sess:
    writer = tf.summary.FileWriter('log/logistic2', sess.graph) # NEW!
    writer.close()

### Even Better: Group Variables Together

In [28]:
tf.reset_default_graph() # Clearing all tensors before this

In [29]:
# Import MINST data
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets('MNIST_data', one_hot=True)

M = 784
K = 10
with tf.name_scope('data'):
    X = tf.placeholder(tf.float32, shape=[None, M], name='Input-Images')
    Y = tf.placeholder(tf.float32, shape=[None, K], name='Output-Labels')
    W = tf.get_variable('Weights', initializer=tf.zeros([M, K]))
    b = tf.get_variable('Bias', initializer=tf.zeros([K]))

with tf.name_scope('model'):
    Yhat = tf.nn.softmax(tf.matmul(X, W) + b, name='Prediction-Labels')

with tf.name_scope('loss'):
    loss = tf.reduce_mean(-tf.reduce_sum(Y*tf.log(Yhat), axis=1), name="Cross-Entropy-Loss")

with tf.name_scope('optimizer'):
    learning_rate = 0.01
    optimizer = tf.train.GradientDescentOptimizer(learning_rate, name="SGD-Optimizer")
    update    = optimizer.minimize(loss)


Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [30]:
# Train
num_epochs  = 25
batch_size  = 100

with tf.Session() as sess:
    writer = tf.summary.FileWriter('log/logistic3', sess.graph) # NEW!
    writer.close()

### Even Even Better: Create Summaries to Track Training

In [31]:
tf.reset_default_graph() # Clearing all tensors before this

In [32]:
# Import MINST data
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets('MNIST_data', one_hot=True)

M = 784
K = 10
with tf.name_scope('data'):
    X = tf.placeholder(tf.float32, shape=[None, M], name='Input-Images')
    Y = tf.placeholder(tf.float32, shape=[None, K], name='Output-Labels')
    W = tf.get_variable('Weights', initializer=tf.zeros([M, K]))
    b = tf.get_variable('Bias', initializer=tf.zeros([K]))

with tf.name_scope('model'):
    Yhat = tf.nn.softmax(tf.matmul(X, W) + b, name='Prediction-Labels')

with tf.name_scope('loss'):
    loss = tf.reduce_mean(-tf.reduce_sum(Y*tf.log(Yhat), axis=1), name="Cross-Entropy-Loss")

with tf.name_scope('optimizer'):
    learning_rate = 0.01
    optimizer = tf.train.GradientDescentOptimizer(learning_rate, name="SGD-Optimizer")
    update    = optimizer.minimize(loss)

with tf.name_scope('summaries'): # NEW!
    tf.summary.scalar('loss', loss)
    tf.summary.histogram('histogram-loss', loss)
    summary_op = tf.summary.merge_all()

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [33]:
# Train
num_epochs  = 25
batch_size  = 100

with tf.Session() as sess:
    writer = tf.summary.FileWriter('log/logistic4', sess.graph)
    
    sess.run(tf.global_variables_initializer())
    total_batch = int(mnist.train.num_examples/batch_size)
    for epoch in range(num_epochs):
        average_cost = 0
        for batch in range(total_batch):
            batch_X, batch_Y = mnist.train.next_batch(batch_size)
            _, c = sess.run([update, loss], feed_dict={X: batch_X,
                                                       Y: batch_Y})
            average_cost += c / total_batch
            summary = sess.run(summary_op, feed_dict={X: batch_X,
                                                    Y: batch_Y})
            global_step = epoch*total_batch + batch
            writer.add_summary(summary, global_step=global_step)
        print("Epoch:",epoch,"Cost:",average_cost)
        
        
        
    writer.close()

Epoch: 0 Cost: 1.18415000103
Epoch: 1 Cost: 0.665413012992
Epoch: 2 Cost: 0.552853881121
Epoch: 3 Cost: 0.498638092279
Epoch: 4 Cost: 0.46549880976
Epoch: 5 Cost: 0.442564203848
Epoch: 6 Cost: 0.425497464917
Epoch: 7 Cost: 0.412214421332
Epoch: 8 Cost: 0.40141326273
Epoch: 9 Cost: 0.392426092245
Epoch: 10 Cost: 0.384764256748
Epoch: 11 Cost: 0.378108848442
Epoch: 12 Cost: 0.3723855522
Epoch: 13 Cost: 0.367273085605
Epoch: 14 Cost: 0.362724641453
Epoch: 15 Cost: 0.358594706519
Epoch: 16 Cost: 0.354899908684
Epoch: 17 Cost: 0.351458825957
Epoch: 18 Cost: 0.348327035796
Epoch: 19 Cost: 0.345387803154
Epoch: 20 Cost: 0.342760479911
Epoch: 21 Cost: 0.340282042704
Epoch: 22 Cost: 0.337932195609
Epoch: 23 Cost: 0.335765975389
Epoch: 24 Cost: 0.333698157641


That's it! A model like logistic regression might be too simple to need a visualization. But imagine you're training a new neural network model architecture you came up with. How would you know if your model is doing well, or that there aren't any bugs in the code in the first place? TensorBoard is a great way to intuitively understand your model performance.

## Exercise
Nicely visualize the linear regression you implemented in the previous exercise.


In [26]:
tf.reset_default_graph() # Clearing all tensors before this

In [27]:
from sklearn.datasets import load_boston
import pandas as pd
import numpy as np
boston = load_boston()
df = pd.DataFrame(boston.data)
df.columns = boston.feature_names
df['PRICE'] = boston.target

M = df.shape[1]
N = df.shape[0]
with tf.name_scope('data'):
    X = tf.placeholder(tf.float32, shape=[None, M],name="House-Features")
    Y = tf.placeholder(tf.float32, shape=[None, 1],name="Housing-Price")
    W = tf.Variable(tf.ones([M, 1]),name="Weights")
with tf.name_scope('model'):
    Yhat = tf.matmul(X,W,name="Price-Predictions")
with tf.name_scope('loss'):
    loss = tf.reduce_mean((Y-Yhat)**2,name="Loss")
with tf.name_scope('optimizer'):
    optimizer = tf.train.GradientDescentOptimizer(0.01)
    update = optimizer.minimize(loss, name="SGD-Optimizer")

with tf.name_scope('summaries'):
    tf.summary.scalar('loss', loss)
    tf.summary.histogram('histogram-loss', loss)
    summary_op = tf.summary.merge_all()
    
X_np = df.iloc[:,:-1].as_matrix()
X_np = (X_np - np.mean(X_np,axis=0)) / np.std(X_np,axis=0)
X_np = np.hstack((X_np,np.ones([X_np.shape[0],1])))
y_np = df.iloc[:,-1].as_matrix()[:,np.newaxis]
np.random.seed(1)
train_indices = np.random.rand(X_np.shape[0]) < 0.80
X_train = X_np[train_indices]
X_test  = X_np[~train_indices]
y_train = y_np[train_indices]
y_test  = y_np[~train_indices]

In [28]:
num_epochs  = 1000
losses = []
with tf.Session() as sess:
    writer = tf.summary.FileWriter('log/linear', sess.graph)
    sess.run(tf.global_variables_initializer())
    for epoch in range(num_epochs):
        _, c, summary = sess.run([update, loss, summary_op], feed_dict={X: X_train, 
                                                      Y: y_train})
        losses.append(c)
        print("Epoch:",epoch,"Cost:",c)
        writer.add_summary(summary, global_step=epoch)
    test_loss = sess.run(loss, feed_dict={X: X_test, 
                                          Y: y_test})
    print("Test Loss:",test_loss)

Epoch: 0 Cost: 604.989
Epoch: 1 Cost: 566.648
Epoch: 2 Cost: 533.349
Epoch: 3 Cost: 504.086
Epoch: 4 Cost: 478.079
Epoch: 5 Cost: 454.72
Epoch: 6 Cost: 433.534
Epoch: 7 Cost: 414.152
Epoch: 8 Cost: 396.281
Epoch: 9 Cost: 379.692
Epoch: 10 Cost: 364.203
Epoch: 11 Cost: 349.67
Epoch: 12 Cost: 335.978
Epoch: 13 Cost: 323.032
Epoch: 14 Cost: 310.756
Epoch: 15 Cost: 299.089
Epoch: 16 Cost: 287.978
Epoch: 17 Cost: 277.381
Epoch: 18 Cost: 267.259
Epoch: 19 Cost: 257.581
Epoch: 20 Cost: 248.321
Epoch: 21 Cost: 239.453
Epoch: 22 Cost: 230.957
Epoch: 23 Cost: 222.812
Epoch: 24 Cost: 215.002
Epoch: 25 Cost: 207.51
Epoch: 26 Cost: 200.322
Epoch: 27 Cost: 193.424
Epoch: 28 Cost: 186.802
Epoch: 29 Cost: 180.446
Epoch: 30 Cost: 174.343
Epoch: 31 Cost: 168.483
Epoch: 32 Cost: 162.856
Epoch: 33 Cost: 157.452
Epoch: 34 Cost: 152.263
Epoch: 35 Cost: 147.278
Epoch: 36 Cost: 142.491
Epoch: 37 Cost: 137.893
Epoch: 38 Cost: 133.476
Epoch: 39 Cost: 129.233
Epoch: 40 Cost: 125.157
Epoch: 41 Cost: 121.242
Epoch

Epoch: 414 Cost: 22.4242
Epoch: 415 Cost: 22.4222
Epoch: 416 Cost: 22.4201
Epoch: 417 Cost: 22.4181
Epoch: 418 Cost: 22.4161
Epoch: 419 Cost: 22.4141
Epoch: 420 Cost: 22.4121
Epoch: 421 Cost: 22.4101
Epoch: 422 Cost: 22.4082
Epoch: 423 Cost: 22.4062
Epoch: 424 Cost: 22.4043
Epoch: 425 Cost: 22.4023
Epoch: 426 Cost: 22.4004
Epoch: 427 Cost: 22.3985
Epoch: 428 Cost: 22.3966
Epoch: 429 Cost: 22.3948
Epoch: 430 Cost: 22.3929
Epoch: 431 Cost: 22.391
Epoch: 432 Cost: 22.3892
Epoch: 433 Cost: 22.3874
Epoch: 434 Cost: 22.3856
Epoch: 435 Cost: 22.3837
Epoch: 436 Cost: 22.3819
Epoch: 437 Cost: 22.3802
Epoch: 438 Cost: 22.3784
Epoch: 439 Cost: 22.3766
Epoch: 440 Cost: 22.3749
Epoch: 441 Cost: 22.3731
Epoch: 442 Cost: 22.3714
Epoch: 443 Cost: 22.3697
Epoch: 444 Cost: 22.368
Epoch: 445 Cost: 22.3663
Epoch: 446 Cost: 22.3646
Epoch: 447 Cost: 22.3629
Epoch: 448 Cost: 22.3612
Epoch: 449 Cost: 22.3596
Epoch: 450 Cost: 22.3579
Epoch: 451 Cost: 22.3563
Epoch: 452 Cost: 22.3547
Epoch: 453 Cost: 22.3531
Ep

Epoch: 789 Cost: 22.1084
Epoch: 790 Cost: 22.1082
Epoch: 791 Cost: 22.1079
Epoch: 792 Cost: 22.1076
Epoch: 793 Cost: 22.1073
Epoch: 794 Cost: 22.107
Epoch: 795 Cost: 22.1067
Epoch: 796 Cost: 22.1065
Epoch: 797 Cost: 22.1062
Epoch: 798 Cost: 22.1059
Epoch: 799 Cost: 22.1056
Epoch: 800 Cost: 22.1054
Epoch: 801 Cost: 22.1051
Epoch: 802 Cost: 22.1048
Epoch: 803 Cost: 22.1045
Epoch: 804 Cost: 22.1043
Epoch: 805 Cost: 22.104
Epoch: 806 Cost: 22.1037
Epoch: 807 Cost: 22.1035
Epoch: 808 Cost: 22.1032
Epoch: 809 Cost: 22.1029
Epoch: 810 Cost: 22.1027
Epoch: 811 Cost: 22.1024
Epoch: 812 Cost: 22.1022
Epoch: 813 Cost: 22.1019
Epoch: 814 Cost: 22.1016
Epoch: 815 Cost: 22.1014
Epoch: 816 Cost: 22.1011
Epoch: 817 Cost: 22.1009
Epoch: 818 Cost: 22.1006
Epoch: 819 Cost: 22.1004
Epoch: 820 Cost: 22.1001
Epoch: 821 Cost: 22.0999
Epoch: 822 Cost: 22.0996
Epoch: 823 Cost: 22.0994
Epoch: 824 Cost: 22.0991
Epoch: 825 Cost: 22.0989
Epoch: 826 Cost: 22.0986
Epoch: 827 Cost: 22.0984
Epoch: 828 Cost: 22.0981
Ep